In [1]:
import sys
import os

# Get the current working directory
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
print(parent_directory)

if 'TimeXer' in parent_directory:
    # Add the parent directory to sys.path
    sys.path.append(parent_directory)
    os.chdir(parent_directory)
%load_ext autoreload
%autoreload 2



/home/robers/projects/TimeXer


In [2]:
from utils.database import connectionSQL
import pandas as pd

engine = connectionSQL("PROMART_PRD", 'cfg/secrets.yaml')

# query = """
# SELECT DISTINCT [Type_query]
# FROM [PROMART_PRD].[dbo].[t_ML_meteoswiss_weather_forecast];"""

# m_suisse_types = pd.read_sql_query(query, con=engine)

# query = """
# SELECT DISTINCT [Type_query]
# FROM [PROMART_PRD].[dbo].[t_ML_weather_data];"""

# weather_data_types = pd.read_sql_query(query, con=engine)

In [3]:
# print(m_suisse_types['Type_query'])
# print(weather_data_types['Type_query'])
m_suisse_type = 'icon_ch2_0'
weather_data_type = 'matics-mos-flex'

In [5]:
Abbreviations = ['ABO', 'PAY']
start_date = '2024-01-01'
end_date = '2024-01-31'

# Convert the list of abbreviations into a SQL-compatible string
abbreviation_list = "', '".join(Abbreviations)

query = f"""
SELECT
      [Abbreviation],
      [Delivery_date],
      [Issue_date],
      [Type_query],
      [rain {{mm/h}}],
      [temperature {{Celsius}}],
      [relative humidity{{%}}],
      [radiation globale {{W/m2}}]
  FROM [PROMART_PRD].[dbo].[t_ML_weather_data]
  WHERE [Type_query] = '{weather_data_type}' 
    AND [Abbreviation] IN ('{abbreviation_list}')
    AND [Delivery_date] BETWEEN '{start_date}' AND '{end_date}'
"""

# Execute the query and load the results into a DataFrame
df = pd.read_sql_query(query, con=engine)
df.drop('Type_query', axis=1, inplace=True)
print(len(df))


68357


In [6]:
# endogenous variables: load from gb 
# exogenous variables: temperature, rain, radiation, calendar_date, day_of_week, is_holiday

from utils.dataset import remove_duplicates_keep_latest
print(len(df['Delivery_date'].unique()), len(df['Issue_date'].unique()))
df_unique = remove_duplicates_keep_latest(df, group_cols=['Abbreviation', 'Delivery_date'], sort_col='Issue_date')
print(len(df_unique['Delivery_date'].unique()), len(df_unique['Issue_date'].unique()))

2881 73
2881 61


In [7]:
from utils.dataset import append_abbreviation_to_columns
# Example columns to update
columns_to_update = ['rain {mm/h}', 'temperature {Celsius}', 'relative humidity{%}', 'radiation globale {W/m2}']

# Apply the function to the dataframe
df_unique_noabbr = append_abbreviation_to_columns(df_unique, columns_to_update)

In [8]:
print(len(df_unique_noabbr))
print(len(df_unique_noabbr['Delivery_date'].unique()))
df_unique_noabbr.columns

2881
2881


Index(['Delivery_date', 'ABO rain {mm/h}', 'ABO temperature {Celsius}',
       'ABO relative humidity{%}', 'ABO radiation globale {W/m2}',
       'PAY rain {mm/h}', 'PAY temperature {Celsius}',
       'PAY relative humidity{%}', 'PAY radiation globale {W/m2}'],
      dtype='object')

In [9]:
query = f"""
SELECT
      [Delivery_date],
      [Issue_date],
      [Bilan],
      [Validation]
FROM [PROMART_PRD].[dbo].[t_ML_validation_gb_consumption] 
WHERE ([Validation] = '100' OR [Validation] = '110')
  AND [Delivery_date] BETWEEN '{start_date}' AND '{end_date}'"""

df_load = pd.read_sql_query(query, con=engine)
df_load_unique = remove_duplicates_keep_latest(df_load, group_cols=['Delivery_date'], sort_col='Issue_date')


In [10]:
# Ensure both dataframes are sorted by 'Delivery_date'
df_unique_noabbr = df_unique_noabbr.sort_values('Delivery_date')
df_load_unique = df_load_unique.sort_values('Delivery_date')

# Perform an inner merge to keep only matching 'Delivery_date' rows in both dataframes
df_unique_merged = pd.merge(
    df_unique_noabbr, 
    df_load_unique[['Delivery_date', 'Bilan']], 
    on='Delivery_date', 
    how='inner'
)

print(len(df_unique_merged))

2881


In [11]:
from utils.dataset import detect_diff_data
df_unique.set_index('Delivery_date')
df_unique_merged.set_index('Delivery_date')

detect_diff_data(df_unique, df_unique_merged, ['ABO', 'PAY'], column_name = 'rain {mm/h}')


No differences detected for abbreviation ABO in radiation data.
No differences detected for abbreviation PAY in radiation data.


In [12]:
df_unique_merged.columns

Index(['Delivery_date', 'ABO rain {mm/h}', 'ABO temperature {Celsius}',
       'ABO relative humidity{%}', 'ABO radiation globale {W/m2}',
       'PAY rain {mm/h}', 'PAY temperature {Celsius}',
       'PAY relative humidity{%}', 'PAY radiation globale {W/m2}', 'Bilan'],
      dtype='object')

In [13]:
from utils.dataset import load_sheets_from_cc_auto

sheets_to_load = {'FR': 'const_fribourg'}
file_path = '/data/www/07_Trading/CourtTerme/Optimisation/Consommation/CourtTerme/dev/opt_cc_auto.xlsm'

df_auto_cc = load_sheets_from_cc_auto(file_path, sheets_to_load)

In [14]:
df_auto_cc.index

DatetimeIndex(['2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04',
               '2010-01-05', '2010-01-06', '2010-01-07', '2010-01-08',
               '2010-01-09', '2010-01-10',
               ...
               '2025-12-22', '2025-12-23', '2025-12-24', '2025-12-25',
               '2025-12-26', '2025-12-27', '2025-12-28', '2025-12-29',
               '2025-12-30', '2025-12-31'],
              dtype='datetime64[ns]', name='Datum', length=5844, freq=None)

In [15]:
from utils.dataset import append_calendar_data
df_unique_merged_cal = append_calendar_data(df_unique_merged, df_auto_cc, '15min')
len(df_unique_merged_cal)

weather index: 2024-01-01 00:00:00 - 2024-01-31 00:00:00
calendar index: 2024-01-01 00:00:00 - 2024-01-31 00:00:00


2881

In [21]:
from utils.dataset import resample_data

df_unique_merged_cal_resampled = resample_data(df_unique_merged_cal, '15min')
print(df_unique_merged_cal_resampled)

                     ABO rain {mm/h}  ABO temperature {Celsius}  \
Delivery_date                                                     
2024-01-01 00:00:00              0.0                        1.0   
2024-01-01 00:15:00              0.0                        0.8   
2024-01-01 00:30:00              0.0                        0.7   
2024-01-01 00:45:00              0.0                        0.5   
2024-01-01 01:00:00              0.0                        0.4   
...                              ...                        ...   
2024-01-30 23:00:00              0.0                        4.6   
2024-01-30 23:15:00              0.0                        4.5   
2024-01-30 23:30:00              0.0                        4.4   
2024-01-30 23:45:00              0.0                        4.4   
2024-01-31 00:00:00              0.0                        4.3   

                     ABO relative humidity{%}  ABO radiation globale {W/m2}  \
Delivery_date                                    

In [24]:
# Save data to csv
from utils.dataset import save_to_csv
save_to_csv(df_unique_merged_cal_resampled, 'dataset/custom/test.csv')

